# Introduction
This part of the repository builds a detection system based on YOLOv4 object detector.

# Environment setup

## Google Drive mount
I'm using Google Colaboratory as my default platform, therefore I need to set up my environment to integrate it with Google Drive. You can skip this bit if you're working locally.

1. Mount Google Drive on the runtime to be able to read and write files. This will ask you to log in to your Google Account and provide an authorization code.
2. Create a symbolic link to a working directory 
3. Change the directory to the one where I cloned my repository.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
!ln -s /content/gdrive/My\ Drive/Colab\ Notebooks/dezeenAI /mydrive
!ls /mydrive

classes.gsheet	       dezeen_2-download.ipynb	files
darknet		       dezeen_3-dataset.ipynb	LICENSE
data		       dezeen_4-model.ipynb	OIDv4_ToolKit
dezeen_0-test.ipynb    dezeen_5-database.ipynb	README.md
dezeen_1-scrape.ipynb  dezeenAI


In [3]:
%cd /mydrive

/content/gdrive/My Drive/Colab Notebooks/dezeenAI


## Libraries & functions
- `cv2` - computer vision library
- `matplotlib.pyplot` - visualisation & plotting library
- `sys` - system-specific parameters and functions
- `os.path` - common pathname manipulations
- `timeit` - cell runtime
- `tqdm` - loop progress bar

In [4]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import os.path
import timeit
import tqdm

# Object detection

## Split dataset
1. Let's divide our dataset into smaller chunks to be able to do the predictions on smaller batches.
2. Save the list of split files into a list to iterate through

In [5]:
%cd /mydrive/files
%mkdir -p tmp

with open('dezeen-images.txt') as fp:
  data = fp.read() 

linklist = data.split('\n')

CHUNK_SIZE = 2000

for n in range(int(len(linklist)/CHUNK_SIZE)+1):
  start = n*CHUNK_SIZE
  end = ((n+1)*CHUNK_SIZE)-1
  if end > len(linklist): 
    end = len(linklist)
  
  filename = 'tmp/dezeen-images-{}.txt'.format(n)
  content = '\n'.join(linklist[start:end])  

  f = open(filename, 'w')
  f.write(content)
  f.close()

/content/gdrive/My Drive/Colab Notebooks/dezeenAI/files


In [6]:
chunks = [] 
DIR = '/mydrive/files/tmp/'
os.chdir(DIR)

for filename in os.listdir(os.getcwd()):
  if filename.startswith('dezeen-images-'):
    chunks.append(DIR + filename)

## Out the box object detection
Darknet is now built and ready to run detections using YOLOv4 and pre-trained weights. It can already detect 80 classes of COCO Dataset. Let's test its performance on a pre-uploaded test picture.

- `./darknet detector test` - YOLOv4 detection function
- `coco.data` - dataset configuration
- `yolov4.cfg` - system architecture configuration
- `yolov4.weights` - pre-trained weights
- `-dont_show` - skips outputting the prediction picture (Notebook shows an error when trying to display the prediction) 
- `-tresh 0.5` - sets the treshold value for predictions
- `dezeen_results.json` - contains saved data
- `dezeen_images.txt` - contains list of images to evaluate

In [7]:
start = timeit.default_timer() # start the times

%cd /mydrive/darknet
!chmod +x ./darknet

for n, chunk in enumerate(chunks):
  input = chunk
  output = '/mydrive/files/tmp/dezeen_results-{}.json'.format(n)
  if os.path.isfile(output):
    print('{}: file already exists! Continuing...'.format(output))
  else:
    !./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show -ext_output -out {output} < {input}

stop = timeit.default_timer() # stop the timer
print('Runtime: {} seconds.'.format(stop-start))

Streaming output truncated to the last 5000 lines.
person: 34%	(left_x:  219   top_y:  186   width:   11   height:   40)
person: 71%	(left_x:  225   top_y:  185   width:   17   height:   42)
person: 43%	(left_x:  242   top_y:  181   width:   17   height:   44)
person: 88%	(left_x:  257   top_y:  181   width:   12   height:   22)
oven: 32%	(left_x:  263   top_y:  244   width:   77   height:  102)
person: 43%	(left_x:  309   top_y:  177   width:   27   height:   32)
person: 93%	(left_x:  344   top_y:  145   width:   84   height:  148)
tie: 32%	(left_x:  361   top_y:  183   width:   44   height:   89)
person: 30%	(left_x:  361   top_y:  183   width:   44   height:   89)
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
/mydrive/data/dezeen/6196/vgch_215_sq.jpg: Predicted in 54.769000 milli-seconds.
car: 50%	(left_x:  157   top_y:  289   width:  295   height:  159)
chair: 29%	(left_x:  182   top_y:  317   width:   26 